# E-Ticaret Müşteri Davranış Analizi ve Satış Tahmini

## Proje Hakkında
Bu notebook, e-ticaret verilerini kullanarak müşteri davranış analizi, segmentasyon, satış tahmini ve müşteri sınıflandırması gerçekleştirmektedir.

### İçerik:
1. **Veri Yükleme ve Hazırlık**
2. **Keşifsel Veri Analizi (EDA)**
3. **Veri Ön İşleme**
4. **Müşteri Segmentasyonu (Kümeleme)**
5. **Satış Tahmini (Regresyon)**
6. **Müşteri Davranış Sınıflandırması**
7. **Sonuçlar ve Öneriler**

---

## 1. Kütüphaneler ve Veri Yükleme

Bu bölümde projemiz için gerekli Python kütüphanelerini yüklüyor ve veri setlerini okuyoruz.

In [ ]:
# Gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Matplotlib best practice setup
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(" Tüm kütüphaneler başarıyla yüklendi!")

In [ ]:
# Veri setlerinin yüklenmesi
try:
    # E-ticaret veri setleri
    customers_df = pd.read_csv('data/customers.csv')
    products_df = pd.read_csv('data/products.csv')
    transactions_df = pd.read_csv('data/transactions.csv')
    interactions_df = pd.read_csv('data/interactions.csv')
    
    print(" Tüm veri setleri başarıyla yüklendi!")
    
    # Veri seti bilgileri
    datasets = {
        'Müşteriler': customers_df,
        'Ürünler': products_df,
        'İşlemler': transactions_df,
        'Etkileşimler': interactions_df
    }
    
    for name, df in datasets.items():
        print(f"\n {name} Dataset:")
        print(f"  - Boyut: {df.shape}")
        print(f"  - Sütunlar: {list(df.columns)}")
        
except FileNotFoundError as e:
    print(f" Veri dosyası bulunamadı: {e}")
    print("Örnek veri oluşturuluyor...")
    
    # Örnek veri oluşturma fonksiyonu
    def create_sample_data():
        np.random.seed(42)
        
        # Müşteri verisi
        customers_data = {
            'customer_id': range(1, 1001),
            'age': np.random.randint(18, 80, 1000),
            'gender': np.random.choice(['M', 'F'], 1000),
            'city': np.random.choice(['İstanbul', 'Ankara', 'İzmir', 'Bursa'], 1000),
            'registration_date': pd.date_range('2020-01-01', periods=1000, freq='D'),
            'total_spent': np.random.uniform(100, 5000, 1000),
            'purchase_frequency': np.random.randint(1, 50, 1000)
        }
        
        # Ürün verisi
        products_data = {
            'product_id': range(1, 501),
            'product_name': [f'Ürün_{i}' for i in range(1, 501)],
            'category': np.random.choice(['Elektronik', 'Giyim', 'Ev & Yaşam', 'Kitap', 'Spor'], 500),
            'price': np.random.uniform(10, 1000, 500),
            'brand': np.random.choice(['Marka_A', 'Marka_B', 'Marka_C', 'Marka_D'], 500)
        }
        
        # İşlem verisi
        transactions_data = {
            'transaction_id': range(1, 5001),
            'customer_id': np.random.randint(1, 1001, 5000),
            'product_id': np.random.randint(1, 501, 5000),
            'quantity': np.random.randint(1, 10, 5000),
            'price': np.random.uniform(10, 1000, 5000),
            'discount': np.random.uniform(0, 0.3, 5000),
            'transaction_date': pd.date_range('2023-01-01', periods=5000, freq='6H'),
            'payment_method': np.random.choice(['Kredi Kartı', 'Debit Kart', 'Havale/EFT', 'Kapıda Ödeme'], 5000)
        }
        
        # Etkileşim verisi
        interactions_data = {
            'interaction_id': range(1, 10001),
            'customer_id': np.random.randint(1, 1001, 10000),
            'product_id': np.random.randint(1, 501, 10000),
            'interaction_type': np.random.choice(['view', 'click', 'add_to_cart', 'wishlist'], 10000),
            'interaction_date': pd.date_range('2023-01-01', periods=10000, freq='3H')
        }
        
        return (
            pd.DataFrame(customers_data),
            pd.DataFrame(products_data),
            pd.DataFrame(transactions_data),
            pd.DataFrame(interactions_data)
        )
    
    # Örnek verileri oluştur
    customers_df, products_df, transactions_df, interactions_df = create_sample_data()
    print(" Örnek veriler oluşturuldu!")
    
    # DataFrame'leri kaydet
    customers_df.to_csv('data/customers.csv', index=False)
    products_df.to_csv('data/products.csv', index=False)
    transactions_df.to_csv('data/transactions.csv', index=False)
    interactions_df.to_csv('data/interactions.csv', index=False)

## 2. Keşifsel Veri Analizi (EDA)

Bu bölümde veri setlerimizi detaylı olarak inceleyerek temel istatistikleri, dağılımları ve ilişkileri analiz edeceğiz.

In [ ]:
# Veri setlerinin genel bilgileri
print("=== VERİ SETİ GENEL BİLGİLERİ ===\n")

datasets_info = {
    'Müşteriler': customers_df,
    'Ürünler': products_df,
    'İşlemler': transactions_df,
    'Etkileşimler': interactions_df
}

for name, df in datasets_info.items():
    print(f" {name} Dataset:")
    print(f"  Boyut: {df.shape}")
    print(f"  Bellek Kullanımı: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"  Eksik Değerler: {df.isnull().sum().sum()}")
    print(f"  Sütunlar: {list(df.columns)}")
    print("-" * 50)

In [ ]:
# Temel istatistikler - Müşteri verisi
print("=== MÜŞTERİ VERİSİ TEMEL İSTATİSTİKLERİ ===")
print("\n Sayısal Değişkenler:")
print(customers_df.describe())

print("\n Kategorik Değişkenler:")
print("\nCinsiyet Dağılımı:")
print(customers_df['gender'].value_counts())

print("\nŞehir Dağılımı:")
print(customers_df['city'].value_counts())

In [ ]:
# Müşteri demografik analizi
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Müşteri Demografik Analizi', fontsize=16, fontweight='bold')

# Yaş dağılımı
axes[0,0].hist(customers_df['age'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Yaş Dağılımı')
axes[0,0].set_xlabel('Yaş')
axes[0,0].set_ylabel('Frekans')

# Cinsiyet dağılımı
gender_counts = customers_df['gender'].value_counts()
axes[0,1].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
axes[0,1].set_title('Cinsiyet Dağılımı')

# Şehir dağılımı
city_counts = customers_df['city'].value_counts()
axes[1,0].bar(city_counts.index, city_counts.values, color='lightcoral')
axes[1,0].set_title('Şehir Dağılımı')
axes[1,0].set_xlabel('Şehir')
axes[1,0].set_ylabel('Müşteri Sayısı')
axes[1,0].tick_params(axis='x', rotation=45)

# Toplam harcama dağılımı
axes[1,1].hist(customers_df['total_spent'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1,1].set_title('Toplam Harcama Dağılımı')
axes[1,1].set_xlabel('Toplam Harcama (₺)')
axes[1,1].set_ylabel('Frekans')

plt.tight_layout()
plt.show()

# Korelasyon matrisi
plt.figure(figsize=(10, 8))
numeric_cols = customers_df.select_dtypes(include=[np.number]).columns
correlation_matrix = customers_df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Müşteri Verisi Korelasyon Matrisi')
plt.tight_layout()
plt.show()

In [ ]:
# Ürün analizi
print("=== ÜRÜN VERİSİ ANALİZİ ===")
print("\n Kategori Dağılımı:")
print(products_df['category'].value_counts())

print("\n Fiyat İstatistikleri:")
print(products_df['price'].describe())

# Ürün kategorileri ve fiyat analizi
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Kategori dağılımı
category_counts = products_df['category'].value_counts()
axes[0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
axes[0].set_title('Ürün Kategorisi Dağılımı')

# Kategoriye göre fiyat dağılımı
products_df.boxplot(column='price', by='category', ax=axes[1])
axes[1].set_title('Kategoriye Göre Fiyat Dağılımı')
axes[1].set_xlabel('Kategori')
axes[1].set_ylabel('Fiyat (₺)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# İşlem verisi analizi
print("=== İŞLEM VERİSİ ANALİZİ ===")

# Tarih formatını düzenle
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])
transactions_df['month'] = transactions_df['transaction_date'].dt.month
transactions_df['day_of_week'] = transactions_df['transaction_date'].dt.dayofweek
transactions_df['hour'] = transactions_df['transaction_date'].dt.hour

print("\n Ödeme Yöntemi Dağılımı:")
print(transactions_df['payment_method'].value_counts())

print("\n Miktar İstatistikleri:")
print(transactions_df['quantity'].describe())

# İşlem trendleri
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('İşlem Verisi Analizi', fontsize=16)

# Ödeme yöntemi dağılımı
payment_counts = transactions_df['payment_method'].value_counts()
axes[0,0].pie(payment_counts.values, labels=payment_counts.index, autopct='%1.1f%%')
axes[0,0].set_title('Ödeme Yöntemi Dağılımı')

# Aylık satış trendi
monthly_sales = transactions_df.groupby('month')['price'].sum()
axes[0,1].plot(monthly_sales.index, monthly_sales.values, marker='o')
axes[0,1].set_title('Aylık Satış Trendi')
axes[0,1].set_xlabel('Ay')
axes[0,1].set_ylabel('Toplam Satış (₺)')

# Haftanın günlerine göre satış
daily_sales = transactions_df.groupby('day_of_week')['price'].sum()
day_names = ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']
axes[1,0].bar(range(7), daily_sales.values)
axes[1,0].set_xticks(range(7))
axes[1,0].set_xticklabels([d[:3] for d in day_names])
axes[1,0].set_title('Haftanın Günlerine Göre Satış')
axes[1,0].set_ylabel('Toplam Satış (₺)')

# Saatlik satış dağılımı
hourly_sales = transactions_df.groupby('hour')['price'].sum()
axes[1,1].plot(hourly_sales.index, hourly_sales.values, marker='o')
axes[1,1].set_title('Saatlik Satış Dağılımı')
axes[1,1].set_xlabel('Saat')
axes[1,1].set_ylabel('Toplam Satış (₺)')

plt.tight_layout()
plt.show()

In [ ]:
# Etkileşim verisi analizi
print("=== ETKİLEŞİM VERİSİ ANALİZİ ===")

# Etkileşim tarihlerini düzenle
interactions_df['interaction_date'] = pd.to_datetime(interactions_df['interaction_date'])
interactions_df['interaction_hour'] = interactions_df['interaction_date'].dt.hour

print("\n Etkileşim Türü Dağılımı:")
interaction_counts = interactions_df['interaction_type'].value_counts()
print(interaction_counts)

# Etkileşim analizi görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Etkileşim türü dağılımı
axes[0].pie(interaction_counts.values, labels=interaction_counts.index, autopct='%1.1f%%')
axes[0].set_title('Etkileşim Türü Dağılımı')

# Saatlik etkileşim dağılımı
hourly_interactions = interactions_df.groupby('interaction_hour').size()
axes[1].plot(hourly_interactions.index, hourly_interactions.values, marker='o', color='orange')
axes[1].set_title('Saatlik Etkileşim Dağılımı')
axes[1].set_xlabel('Saat')
axes[1].set_ylabel('Etkileşim Sayısı')

plt.tight_layout()
plt.show()

## 3. Veri Ön İşleme

Bu bölümde verilerimizi modelleme için hazırlayacağız: eksik değerleri doldurma, aykırı değerleri tespit etme, özellik mühendisliği ve normalizasyon işlemleri.

In [ ]:
# Eksik değer analizi
print("=== EKSİK DEĞER ANALİZİ ===\n")

for name, df in datasets_info.items():
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    
    if missing_values.sum() > 0:
        print(f" {name} - Eksik Değerler:")
        for col in df.columns:
            if missing_values[col] > 0:
                print(f"  {col}: {missing_values[col]} ({missing_percent[col]:.1f}%)")
    else:
        print(f" {name} - Eksik değer yok")
    print("-" * 30)

In [ ]:
# Eksik değerleri doldurma (eğer varsa)
def handle_missing_values(df):
    """Eksik değerleri uygun stratejilerle doldur"""
    df_filled = df.copy()
    
    # Sayısal değişkenler için median
    numeric_cols = df_filled.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_filled[col].isnull().sum() > 0:
            df_filled[col].fillna(df_filled[col].median(), inplace=True)
    
    # Kategorik değişkenler için mode
    categorical_cols = df_filled.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df_filled[col].isnull().sum() > 0:
            df_filled[col].fillna(df_filled[col].mode()[0], inplace=True)
    
    return df_filled

# Eksik değerleri işle
customers_df = handle_missing_values(customers_df)
products_df = handle_missing_values(products_df)
transactions_df = handle_missing_values(transactions_df)
interactions_df = handle_missing_values(interactions_df)

print(" Eksik değerler dolduruldu!")

In [ ]:
# Aykırı değer tespiti ve temizleme
def detect_outliers(df, column, method='iqr'):
    """Aykırı değerleri tespit et"""
    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    elif method == 'zscore':
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        outliers = df[z_scores > 3]
    
    return outliers

# Aykırı değer analizi
print("=== AYKIRI DEĞER ANALİZİ ===\n")

# Sayısal değişkenler için aykırı değer kontrolü
numeric_columns = ['age', 'total_spent', 'purchase_frequency', 'price', 'quantity', 'discount']

for col in numeric_columns:
    if col in transactions_df.columns or col in customers_df.columns or col in products_df.columns:
        # Uygun dataframe'i seç
        if col in customers_df.columns:
            df = customers_df
        elif col in transactions_df.columns:
            df = transactions_df
        elif col in products_df.columns:
            df = products_df
        else:
            continue
            
        outliers = detect_outliers(df, col)
        print(f" {col}: {len(outliers)} aykırı değer tespit edildi")
        
        if len(outliers) > 0:
            # Aykırı değerleri görselleştir
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 2, 1)
            plt.boxplot(df[col])
            plt.title(f'{col} - Box Plot')
            plt.ylabel(col)
            
            plt.subplot(1, 2, 2)
            plt.hist(df[col], bins=30, alpha=0.7)
            plt.title(f'{col} - Histogram')
            plt.xlabel(col)
            plt.ylabel('Frekans')
            
            plt.tight_layout()
            plt.show()

In [ ]:
# Özellik Mühendisliği
print("=== ÖZELLİK MÜHENDİSLİĞİ ===\n")

# 1. RFM Analizi (Recency, Frequency, Monetary)
def calculate_rfm(df_transactions, df_customers):
    """RFM analizini hesapla"""
    
    # En son işlem tarihi
    max_date = df_transactions['transaction_date'].max()
    
    # Müşteri başına RFM hesapla
    rfm = df_transactions.groupby('customer_id').agg({
        'transaction_date': 'max',  # Recency: Son işlem tarihi
        'transaction_id': 'count',  # Frequency: İşlem sayısı
        'price': 'sum'              # Monetary: Toplam harcama
    }).reset_index()
    
    rfm.columns = ['customer_id', 'last_purchase_date', 'frequency', 'monetary']
    
    # Recency'yi gün olarak hesapla
    rfm['recency'] = (max_date - rfm['last_purchase_date']).dt.days
    
    # Müşteri verileriyle birleştir
    rfm = rfm.merge(df_customers[['customer_id', 'age', 'gender', 'city']], on='customer_id', how='left')
    
    return rfm

# RFM hesapla
rfm_data = calculate_rfm(transactions_df, customers_df)
print("RFM Analizi tamamlandı!")
print(f"RFM verisi boyutu: {rfm_data.shape}")
print("\nRFM İstatistikleri:")
print(rfm_data[['recency', 'frequency', 'monetary']].describe())

In [ ]:
# 2. Müşteri Yaşam Boyu Değeri (CLV) Hesaplama
def calculate_clv(rfm_df, discount_rate=0.01):
    """Müşteri Yaşam Boyu Değerini hesapla"""
    
    # Ortalama satın alma sıklığı (aylık)
    avg_purchase_frequency = rfm_df['frequency'] / 12  # 12 ay varsayımı
    
    # Ortalama sipariş değeri
    avg_order_value = rfm_df['monetary'] / rfm_df['frequency']
    
    # Yaşam boyu değer (basit formül)
    clv = avg_order_value * avg_purchase_frequency * (1 / discount_rate)
    
    rfm_df['clv'] = clv
    rfm_df['avg_order_value'] = avg_order_value
    rfm_df['avg_purchase_frequency'] = avg_purchase_frequency
    
    return rfm_df

# CLV hesapla
rfm_data = calculate_clv(rfm_data)

print(" CLV Hesaplama tamamlandı!")
print("\nCLV İstatistikleri:")
print(rfm_data[['clv', 'avg_order_value', 'avg_purchase_frequency']].describe())

In [ ]:
# 3. Müşteri segmentasyonu için kategorik değişkenleri encode etme
def encode_categorical_features(df):
    """Kategorik değişkenleri encode et"""
    df_encoded = df.copy()
    
    # Gender encoding
    df_encoded['gender_encoded'] = df_encoded['gender'].map({'M': 1, 'F': 0})
    
    # City encoding (Label Encoding)
    le_city = LabelEncoder()
    df_encoded['city_encoded'] = le_city.fit_transform(df_encoded['city'])
    
    return df_encoded, le_city

# Kategorik değişkenleri encode et
rfm_encoded, city_encoder = encode_categorical_features(rfm_data)

print(" Kategorik değişkenler encode edildi!")
print(f"\nŞehir encoding haritası:")
for i, city in enumerate(city_encoder.classes_):
    print(f"  {city}: {i}")

In [ ]:
# 4. Veri normalizasyonu/ölçekleme
def scale_features(df, feature_columns):
    """Özellikleri ölçekle"""
    scaler = StandardScaler()
    df_scaled = df.copy()
    
    # Ölçeklenecek özellikler
    scaled_features = scaler.fit_transform(df[feature_columns])
    
    # Scaled features'ları DataFrame'e ekle
    for i, col in enumerate(feature_columns):
        df_scaled[f'{col}_scaled'] = scaled_features[:, i]
    
    return df_scaled, scaler

# Ölçeklenecek özellikler
features_to_scale = ['age', 'recency', 'frequency', 'monetary', 'clv', 'avg_order_value']

# Özellikleri ölçekle
rfm_scaled, feature_scaler = scale_features(rfm_encoded, features_to_scale)

print(" Veri normalizasyonu tamamlandı!")
print("\nÖlçeklenmiş özelliklerin istatistikleri:")
print(rfm_scaled[[f'{col}_scaled' for col in features_to_scale]].describe())

In [ ]:
# Hazırlanan verinin final kontrolü
print("=== HAZIRLANAN VERİ KONTROLÜ ===\n")

print(f"Final RFM verisi boyutu: {rfm_scaled.shape}")
print(f"Sütunlar: {list(rfm_scaled.columns)}")
print(f"Eksik değerler: {rfm_scaled.isnull().sum().sum()}")

print("\nSon birkaç satır:")
print(rfm_scaled.head())

# Final veri setini kaydet
rfm_scaled.to_csv('data/processed_rfm_data.csv', index=False)
print("\n İşlenmiş veri kaydedildi: data/processed_rfm_data.csv")

## 4. Müşteri Segmentasyonu (Kümeleme)

Bu bölümde K-Means algoritması kullanarak müşterileri segmentlere ayıracağız ve optimal cluster sayısını belirleyeceğiz.

In [ ]:
# Optimal cluster sayısını belirleme
print("=== KÜMELEME ANALİZİ ===\n")

# Kümeleme için özellikler
clustering_features = ['recency_scaled', 'frequency_scaled', 'monetary_scaled', 'clv_scaled', 'age_scaled']
X_clustering = rfm_scaled[clustering_features]

print(f"Kümeleme için kullanılacak özellikler: {clustering_features}")
print(f"Veri boyutu: {X_clustering.shape}")

# Elbow Method
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_clustering)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_clustering, kmeans.labels_))

# Elbow Method görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Elbow plot
ax1.plot(K_range, inertias, marker='o')
ax1.set_title('Elbow Method (Within-Cluster Sum of Squares)')
ax1.set_xlabel('Cluster Sayısı (k)')
ax1.set_ylabel('WCSS')
ax1.grid(True)

# Silhouette score plot
ax2.plot(K_range, silhouette_scores, marker='s', color='orange')
ax2.set_title('Silhouette Score')
ax2.set_xlabel('Cluster Sayısı (k)')
ax2.set_ylabel('Silhouette Score')
ax2.grid(True)

plt.tight_layout()
plt.show()

# En iyi k değerini bul
best_k = K_range[np.argmax(silhouette_scores)]
print(f"\n En iyi cluster sayısı (Silhouette Score'a göre): {best_k}")
print(f"En yüksek Silhouette Score: {max(silhouette_scores):.3f}")

# Tablo olarak göster
results_df = pd.DataFrame({
    'K': K_range,
    'WCSS': inertias,
    'Silhouette_Score': silhouette_scores
})
print("\nKümeleme Sonuçları:")
print(results_df)

In [ ]:
# Optimal k ile final kümeleme
print(f"=== K={best_k} İLE KÜMELEME ===\n")

# Final K-Means model
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = final_kmeans.fit_predict(X_clustering)

# Cluster etiketlerini veriye ekle
rfm_scaled['cluster'] = cluster_labels

print(f"Küme dağılımı:")
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
for i, count in cluster_counts.items():
    print(f"  Cluster {i}: {count} müşteri (%{count/len(cluster_labels)*100:.1f})")

# Cluster merkezleri
cluster_centers = final_kmeans.cluster_centers_
centers_df = pd.DataFrame(cluster_centers, columns=clustering_features)
centers_df['cluster'] = range(best_k)

print("\nCluster Merkezleri (ölçeklenmiş değerler):")
print(centers_df)

In [ ]:
# Segment analizi ve yorumlama
print("=== SEGMENT ANALİZİ VE YORUMLAMA ===\n")

# Her cluster'ın özelliklerini analiz et
cluster_analysis = rfm_scaled.groupby('cluster').agg({
    'age': 'mean',
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'clv': 'mean',
    'avg_order_value': 'mean'
}).round(2)

print("Cluster Özellikleri:")
print(cluster_analysis)

# Segment isimlendirme ve yorumlama
segment_names = {}
for cluster_id in range(best_k):
    cluster_data = cluster_analysis.loc[cluster_id]
    
    # Segment karakteristiklerine göre isimlendirme
    if cluster_data['recency'] < cluster_analysis['recency'].median() and cluster_data['frequency'] > cluster_analysis['frequency'].median():
        name = "Loyal Customers"
    elif cluster_data['monetary'] > cluster_analysis['monetary'].quantile(0.75):
        name = "High Value Customers"
    elif cluster_data['recency'] > cluster_analysis['recency'].quantile(0.75):
        name = "At Risk Customers"
    elif cluster_data['frequency'] < cluster_analysis['frequency'].quantile(0.25):
        name = "New Customers"
    else:
        name = f"Segment {cluster_id}"
    
    segment_names[cluster_id] = name
    
    print(f"\nCluster {cluster_id}: {name}")
    print(f"   Müşteri Sayısı: {cluster_counts[cluster_id]} (%{cluster_counts[cluster_id]/len(cluster_labels)*100:.1f})")
    print(f"   Ortalama Yaş: {cluster_data['age']:.1f}")
    print(f"   Ortalama Recency: {cluster_data['recency']:.1f} gün")
    print(f"   Ortalama Frequency: {cluster_data['frequency']:.1f}")
    print(f"   Ortalama Monetary: {cluster_data['monetary']:.2f} ₺")
    print(f"   Ortalama CLV: {cluster_data['clv']:.2f} ₺")

# Segment isimlerini veriye ekle
rfm_scaled['segment_name'] = rfm_scaled['cluster'].map(segment_names)

print("\n" + "="*50)
print("SEGMENT ÖZETİ:")
segment_summary = rfm_scaled['segment_name'].value_counts()
print(segment_summary)

In [ ]:
# Segment görselleştirme
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Müşteri Segmentleri Analizi', fontsize=16, fontweight='bold')

# 1. Segment dağılımı (pasta grafik)
segment_counts = rfm_scaled['segment_name'].value_counts()
axes[0,0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%')
axes[0,0].set_title('Müşteri Segment Dağılımı')

# 2. Recency vs Frequency scatter
colors = ['red', 'blue', 'green', 'orange', 'purple'][:best_k]
for i in range(best_k):
    cluster_data = rfm_scaled[rfm_scaled['cluster'] == i]
    axes[0,1].scatter(cluster_data['recency'], cluster_data['frequency'], 
                     c=colors[i], label=segment_names[i], alpha=0.6)
axes[0,1].set_xlabel('Recency (Gün)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Recency vs Frequency')
axes[0,1].legend()

# 3. Monetary vs CLV scatter
for i in range(best_k):
    cluster_data = rfm_scaled[rfm_scaled['cluster'] == i]
    axes[1,0].scatter(cluster_data['monetary'], cluster_data['clv'], 
                     c=colors[i], label=segment_names[i], alpha=0.6)
axes[1,0].set_xlabel('Monetary (₺)')
axes[1,0].set_ylabel('CLV (₺)')
axes[1,0].set_title('Monetary vs CLV')
axes[1,0].legend()

# 4. Segment başına ortalama CLV
avg_clv_by_segment = rfm_scaled.groupby('segment_name')['clv'].mean().sort_values(ascending=False)
axes[1,1].bar(range(len(avg_clv_by_segment)), avg_clv_by_segment.values)
axes[1,1].set_xticks(range(len(avg_clv_by_segment)))
axes[1,1].set_xticklabels(avg_clv_by_segment.index, rotation=45, ha='right')
axes[1,1].set_title('Segment Başına Ortalama CLV')
axes[1,1].set_ylabel('Ortalama CLV (₺)')

plt.tight_layout()
plt.show()

# Silhouette analizi
sample_silhouette_values = silhouette_samples(X_clustering, cluster_labels)

fig, ax = plt.subplots(figsize=(10, 6))
y_lower = 10
for i in range(best_k):
    cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    cluster_silhouette_values.sort()
    
    size_cluster_i = cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = colors[i]
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                     0, cluster_silhouette_values,
                     facecolor=color, edgecolor=color, alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax.set_xlabel('Silhouette coefficient values')
ax.set_ylabel('Cluster label')
ax.set_title('Silhouette Analysis')

ax.axvline(x=np.mean(sample_silhouette_values), color="red", linestyle="--")
plt.show()

## 5. Satış Tahmini (Regresyon)

Bu bölümde müşteri segmentasyonu ve diğer özellikleri kullanarak gelecekteki satışları tahmin edeceğiz.

In [ ]:
# Satış tahmini için veri hazırlama
print("=== SATIŞ TAHMİNİ VERİ HAZIRLAMA ===\n")

# Günlük satış verilerini hazırla
daily_sales = transactions_df.groupby('transaction_date').agg({
    'price': 'sum',
    'quantity': 'sum',
    'transaction_id': 'count'
}).reset_index()

daily_sales.columns = ['date', 'total_sales', 'total_quantity', 'transaction_count']

# Zaman özellikleri ekle
daily_sales['day_of_week'] = daily_sales['date'].dt.dayofweek
daily_sales['month'] = daily_sales['date'].dt.month
daily_sales['day_of_month'] = daily_sales['date'].dt.day
daily_sales['quarter'] = daily_sales['date'].dt.quarter
daily_sales['is_weekend'] = (daily_sales['day_of_week'] >= 5).astype(int)

# Lag özellikleri (gecikmeli değerler)
daily_sales['sales_lag1'] = daily_sales['total_sales'].shift(1)
daily_sales['sales_lag7'] = daily_sales['total_sales'].shift(7)
daily_sales['sales_lag30'] = daily_sales['total_sales'].shift(30)

# Hareketli ortalamalar
daily_sales['sales_ma7'] = daily_sales['total_sales'].rolling(window=7).mean()
daily_sales['sales_ma30'] = daily_sales['total_sales'].rolling(window=30).mean()

# İlk eksik değerleri kaldır
daily_sales = daily_sales.dropna()

print(f" Günlük satış verisi boyutu: {daily_sales.shape}")
print(f"Tarih aralığı: {daily_sales['date'].min()} - {daily_sales['date'].max()}")
print("\nGünlük satış istatistikleri:")
print(daily_sales['total_sales'].describe())

In [ ]:
# Özellik seçimi ve hazırlama
# Hedef değişken: total_sales
target = 'total_sales'

# Özellik seti
feature_columns = [
    'total_quantity', 'transaction_count', 'day_of_week', 'month', 
    'day_of_month', 'quarter', 'is_weekend', 'sales_lag1', 'sales_lag7',
    'sales_lag30', 'sales_ma7', 'sales_ma30'
]

X = daily_sales[feature_columns]
y = daily_sales[target]

print(f"Özellik sayısı: {len(feature_columns)}")
print(f"Örnek sayısı: {len(X)}")
print(f"Hedef değişken: {target}")

# Eğitim/test split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, shuffle=False, random_state=42
)

print(f"\nEğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")

# Özellik ölçekleme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(" Özellik ölçekleme tamamlandı!")

In [ ]:
# Model eğitimi ve değerlendirme
print("=== REGRESYON MODELLERİ ===\n")

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Model sonuçlarını saklayacak liste
results = []

for name, model in models.items():
    print(f"🔧 {name} eğitiliyor...")
    
    # Model eğitimi
    model.fit(X_train_scaled, y_train)
    
    # Tahminler
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Metrikler
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
    cv_mae = -cv_scores.mean()
    
    results.append({
        'Model': name,
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'CV MAE': cv_mae
    })
    
    print(f"   Test MAE: {test_mae:.2f}")
    print(f"   Test RMSE: {test_rmse:.2f}")
    print(f"   Test R²: {test_r2:.3f}")
    print(f"   CV MAE: {cv_mae:.2f}")
    print("-" * 40)

# Sonuçları DataFrame'e çevir
results_df = pd.DataFrame(results)
print("\n MODEL KARŞILAŞTIRMA:")
print(results_df.round(3))

In [ ]:
# En iyi modeli seç ve detaylı analiz
best_model_name = results_df.loc[results_df['Test MAE'].idxmin(), 'Model']
print(f" En iyi model: {best_model_name}")

# En iyi modeli yeniden eğit
best_model = models[best_model_name]
best_model.fit(X_train_scaled, y_train)

# Feature importance (eğer model destekliyorsa)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n {best_model_name} - Feature Importance:")
    print(feature_importance)
    
    # Feature importance görselleştirme
    plt.figure(figsize=(10, 8))
    plt.barh(feature_importance['feature'], feature_importance['importance'])
    plt.title(f'{best_model_name} - Feature Importance')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()

elif hasattr(best_model, 'coef_'):
    feature_coef = pd.DataFrame({
        'feature': feature_columns,
        'coefficient': best_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print(f"\n {best_model_name} - Feature Coefficients:")
    print(feature_coef)

# Tahmin vs gerçek değerler
y_test_pred = best_model.predict(X_test_scaled)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_test_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Gerçek Değerler')
plt.ylabel('Tahmin Edilen Değerler')
plt.title(f'{best_model_name} - Tahmin vs Gerçek')

plt.subplot(1, 2, 2)
residuals = y_test - y_test_pred
plt.scatter(y_test_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin Edilen Değerler')
plt.ylabel('Artıklar (Residuals)')
plt.title('Artık Analizi')

plt.tight_layout()
plt.show()

## 6. Müşteri Davranış Sınıflandırması

Bu bölümde müşterileri loyal/non-loyal olarak sınıflandıracağız ve hangi faktörlerin müşteri sadakatini etkilediğini analiz edeceğiz.

In [ ]:
# Loyal müşteri sınıflandırması için veri hazırlama
print("=== MÜŞTERİ DAVRANIŞ SINIFLANDIRMASI ===\n")

# Loyal müşteri tanımı (frequency ve monetary'e göre)
frequency_threshold = rfm_scaled['frequency'].quantile(0.7)  # Üst %30
monetary_threshold = rfm_scaled['monetary'].quantile(0.7)   # Üst %30

print(f" Loyal Müşteri Eşikleri:")
print(f"  Frequency eşiği: {frequency_threshold:.2f}")
print(f"  Monetary eşiği: {monetary_threshold:.2f}")

# Loyal müşteri etiketi oluştur
rfm_scaled['is_loyal'] = ((rfm_scaled['frequency'] >= frequency_threshold) & 
                          (rfm_scaled['monetary'] >= monetary_threshold)).astype(int)

loyal_counts = rfm_scaled['is_loyal'].value_counts()
print(f"\n Müşteri Dağılımı:")
print(f"  Loyal müşteriler: {loyal_counts[1]} (%{loyal_counts[1]/len(rfm_scaled)*100:.1f})")
print(f"  Non-loyal müşteriler: {loyal_counts[0]} (%{loyal_counts[0]/len(rfm_scaled)*100:.1f})")

# Sınıflandırma için özellikler
classification_features = [
    'age_scaled', 'recency_scaled', 'frequency_scaled', 
    'monetary_scaled', 'clv_scaled', 'avg_order_value_scaled',
    'gender_encoded', 'city_encoded'
]

X_class = rfm_scaled[classification_features]
y_class = rfm_scaled['is_loyal']

print(f"\n Sınıflandırma veri boyutu: {X_class.shape}")
print(f"Özellikler: {classification_features}")

In [ ]:
# Train/test split
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

print(f"Eğitim seti boyutu: {X_train_class.shape}")
print(f"Test seti boyutu: {X_test_class.shape}")
print(f"Eğitim seti loyal oranı: {y_train_class.mean():.3f}")
print(f"Test seti loyal oranı: {y_test_class.mean():.3f}")

# Özellik ölçekleme
class_scaler = StandardScaler()
X_train_class_scaled = class_scaler.fit_transform(X_train_class)
X_test_class_scaled = class_scaler.transform(X_test_class)

print(" Sınıflandırma veri hazırlığı tamamlandı!")

In [ ]:
# Sınıflandırma modelleri
print("=== SINIFLANDIRMA MODELLERİ ===\n")

classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

classification_results = []

for name, clf in classifiers.items():
    print(f" {name} eğitiliyor...")
    
    # Model eğitimi
    clf.fit(X_train_class_scaled, y_train_class)
    
    # Tahminler
    y_train_pred = clf.predict(X_train_class_scaled)
    y_test_pred = clf.predict(X_test_class_scaled)
    
    # Sınıflandırma raporu
    test_report = classification_report(y_test_class, y_test_pred, output_dict=True)
    train_report = classification_report(y_train_class, y_train_pred, output_dict=True)
    
    classification_results.append({
        'Model': name,
        'Train Accuracy': train_report['accuracy'],
        'Test Accuracy': test_report['accuracy'],
        'Test Precision': test_report['1']['precision'],
        'Test Recall': test_report['1']['recall'],
        'Test F1-Score': test_report['1']['f1-score']
    })
    
    print(f"   Test Accuracy: {test_report['accuracy']:.3f}")
    print(f"   Test Precision: {test_report['1']['precision']:.3f}")
    print(f"   Test Recall: {test_report['1']['recall']:.3f}")
    print(f"   Test F1-Score: {test_report['1']['f1-score']:.3f}")
    print("-" * 40)

# Sonuçları DataFrame'e çevir
class_results_df = pd.DataFrame(classification_results)
print("\n SINIFLANDIRMA MODEL KARŞILAŞTIRMA:")
print(class_results_df.round(3))

In [ ]:
# En iyi sınıflandırma modelini seç
best_classifier_name = class_results_df.loc[class_results_df['Test F1-Score'].idxmax(), 'Model']
print(f" En iyi sınıflandırma modeli: {best_classifier_name}")

# En iyi modeli yeniden eğit
best_classifier = classifiers[best_classifier_name]
best_classifier.fit(X_train_class_scaled, y_train_class)

# Confusion Matrix
y_test_pred_class = best_classifier.predict(X_test_class_scaled)
cm = confusion_matrix(y_test_class, y_test_pred_class)

plt.figure(figsize=(12, 5))

# Confusion Matrix Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Loyal', 'Loyal'], 
            yticklabels=['Non-Loyal', 'Loyal'])
plt.title(f'{best_classifier_name} - Confusion Matrix')
plt.ylabel('Gerçek Değer')
plt.xlabel('Tahmin Edilen')

# Classification Report
plt.subplot(1, 2, 2)
report = classification_report(y_test_class, y_test_pred_class, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(3)

# Heatmap olarak göster
sns.heatmap(report_df.iloc[:-1, :-1], annot=True, cmap='RdYlGn')
plt.title('Classification Report')
plt.tight_layout()
plt.show()

print("\n Classification Report:")
print(pd.DataFrame(classification_report(y_test_class, y_test_pred_class, output_dict=True)).round(3))

In [ ]:
# Feature Importance analizi
print("=== FEATURE IMPORTANCE ANALİZİ ===\n")

if hasattr(best_classifier, 'feature_importances_'):
    # Random Forest için feature importance
    feature_importance = pd.DataFrame({
        'feature': classification_features,
        'importance': best_classifier.feature_importances_
    }).sort_values('importance', ascending=False)
    
elif hasattr(best_classifier, 'coef_'):
    # Logistic Regression için feature importance (coefficients)
    feature_importance = pd.DataFrame({
        'feature': classification_features,
        'importance': np.abs(best_classifier.coef_[0])
    }).sort_values('importance', ascending=False)

print(f" {best_classifier_name} - Feature Importance:")
print(feature_importance)

# Feature importance görselleştirme
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title(f'{best_classifier_name} - Feature Importance')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# En önemli özellikler yorumu
print("\n💡 EN ÖNEMLİ ÖZELLİKLER:")
top_features = feature_importance.head(5)
for idx, row in top_features.iterrows():
    feature_name = row['feature'].replace('_scaled', '').replace('_encoded', '')
    print(f"  {idx+1}. {feature_name}: {row['importance']:.3f}")

## 7. Sonuçlar ve Raporlama

Bu bölümde proje sonuçlarını özetleyip iş önerileri ve gelecek çalışmalar için öneriler sunacağız.

In [ ]:
# PROJE SONUÇLARI ÖZETİ
print("=" * 80)
print("                    PROJE SONUÇLARI ÖZETİ")
print("=" * 80)

print("\n 1. VERİ ANALİZİ SONUÇLARI:")
print(f"   • Toplam müşteri sayısı: {len(customers_df):,}")
print(f"   • Toplam ürün sayısı: {len(products_df):,}")
print(f"   • Toplam işlem sayısı: {len(transactions_df):,}")
print(f"   • Toplam etkileşim sayısı: {len(interactions_df):,}")

avg_customer_value = customers_df['total_spent'].mean()
print(f"   • Ortalama müşteri değeri: {avg_customer_value:.2f} ₺")

print("\n 2. MÜŞTERİ SEGMENTASYONU:")
print(f"   • Optimum cluster sayısı: {best_k}")
print(f"   • Segment dağılımı:")
for segment, count in segment_summary.items():
    print(f"     - {segment}: {count} müşteri (%{count/len(rfm_scaled)*100:.1f})")

print("\n 3. SATIŞ TAHMİNİ:")
best_regression = results_df.loc[results_df['Test MAE'].idxmin()]
print(f"   • En iyi model: {best_regression['Model']}")
print(f"   • Test MAE: {best_regression['Test MAE']:.2f}")
print(f"   • Test R²: {best_regression['Test R²']:.3f}")
print(f"   • Test RMSE: {best_regression['Test RMSE']:.2f}")

print("\n 4. MÜŞTERİ SINIFLANDIRMASI:")
best_classification = class_results_df.loc[class_results_df['Test F1-Score'].idxmax()]
print(f"   • En iyi model: {best_classification['Model']}")
print(f"   • Test Accuracy: {best_classification['Test Accuracy']:.3f}")
print(f"   • Test F1-Score: {best_classification['Test F1-Score']:.3f}")
print(f"   • Loyal müşteri oranı: %{y_class.mean()*100:.1f}")

print("\n 5. ÖNE ÇIKAN BULGULAR:")
print(f"   • En değerli segment: {avg_clv_by_segment.index[0]}")
print(f"   • En yüksek CLV: {avg_clv_by_segment.iloc[0]:.2f} ₺")

# Feature importance'dan en önemli faktör
top_factor = feature_importance.iloc[0]['feature'].replace('_scaled', '').replace('_encoded', '')
print(f"   • Müşteri sadakatinde en önemli faktör: {top_factor}")

print("\n" + "=" * 80)

In [ ]:
# İş önerileri ve stratejiler
print("=" * 80)
print("                    İŞ ÖNERİLERİ VE STRATEJİLER")
print("=" * 80)

print("\ 1. MÜŞTERİ SEGMENTASYONU STRATEJİLERİ:")
print("\n    Loyal Customers (%_loyal):")
print("   • Özel müşteri programları oluşturun")
print("   • Early access fırsatları sunun")
print("   • Kişiselleştirilmiş öneriler geliştirin")
print("   • Sadakat ödülleri programları uygulayın")

print("\n    High Value Customers:")
print("   • Premium hizmet paketleri oluşturun")
print("   • VIP müşteri temsilcisi atayın")
print("   • Exclusive ürün lansmanları yapın")
print("   • İndirim ve kampanya önceliği verin")

print("\n     At Risk Customers:")
print("   • Win-back kampanyaları düzenleyin")
print("   • Özel indirimler teklif edin")
print("   • Anketlerle geri bildirim alın")
print("   • Müşteri hizmetleri ile iletişime geçin")

print("\n 2. SATIŞ TAHMİNİ VE İŞ PLANLAMA:")
print("   • Günlük satış tahminlerini kullanarak envanter yönetimini optimize edin")
print("   • Yüksek satış günleri için hazırlık yapın")
print("   • Sezonluk trendleri göz önünde bulundurun")
print("   • Pazarlama bütçesini tahminlere göre planlayın")

print("\n 3. MÜŞTERİ DAVRANIŞ ANALİZİ:")
print(f"   • En önemli faktör ({top_factor}) üzerinde odaklanın")
print("   • Müşteri yolculuğunu optimize edin")
print("   • Kişiselleştirilmiş deneyimler sunun")
print("   • Etkileşim noktalarını güçlendirin")

print("\n  4. GENEL İŞ GELİŞTİRME ÖNERİLERİ:")
print("   • Real-time analitik dashboard oluşturun")
print("   • Müşteri yaşam döngüsü takibini otomatikleştirin")
print("   • AI destekli öneri sistemleri geliştirin")
print("   • Multi-channel entegrasyon sağlayın")
print("   • A/B testleri ile stratejileri optimize edin")

print("\n" + "=" * 80)

In [ ]:
# Gelecek çalışmalar ve iyileştirmeler
print("=" * 80)
print("                 GELECEK ÇALIŞMALAR VE İYİLEŞTİRMELER")
print("=" * 80)

print("\n 1. VERİ BİLİMİ GELİŞTİRMELERİ:")
print("    Veri Genişletme:")
print("   • Daha fazla müşteri demografik verisi toplayın")
print("   • Sosyal medya etkileşim verilerini dahil edin")
print("   • Ürün yorum ve puanlama verilerini kullanın")
print("   • Sesli müşteri hizmetleri kayıtlarını analiz edin")

print("\n    Gelişmiş Modeller:")
print("   • Deep Learning modelleri deneyin (LSTM, GRU)")
print("   • Ensemble yöntemlerini test edin")
print("   • Time series forecasting için Prophet kullanın")
print("   • NLP analizi ile ürün açıklamalarını değerlendirin")

print("\ 2. TEKNİK GELİŞTİRMELER:")
print("    Sistem Optimizasyonu:")
print("   • Real-time scoring sistemi kurun")
print("   • MLOps pipeline'ını otomatikleştirin")
print("   • API servisi geliştirin")
print("   • Cloud tabanlı çözümler implementasyonu")

print("\n    Görselleştirme ve Raporlama:")
print("   • Interactive dashboard geliştirin")
print("   • Automated reporting sistemi")
print("   • Mobile-friendly uygulamalar")
print("   • Executive summary raporları")

print("\n 3. İŞ UYGULAMALARI:")
print("    Operasyonel Entegrasyon:")
print("   • CRM sistemine entegre edin")
print("   • Marketing automation ile bağlantı kurun")
print("   • Inventory management sistemine dahil edin")
print("   • Customer support'ta kullanın")

print("\n    Sürekli İyileştirme:")
print("   • Model performansını düzenli izleyin")
print("   • A/B testing ile validasyon yapın")
print("   • Feedback loop kurun")
print("   • Business impact'ini ölçün")

print("\n  4. ARAŞTIRMA KONULARI:")
print("   • Causal inference analizi")
print("   • Customer churn prediction")
print("   • Market basket analysis")
print("   • Recommendation systems optimization")
print("   • Dynamic pricing strategies")
print("   • Cross-channel customer journey mapping")

print("\n" + "=" * 80)
print("Bu projede elde edilen sonuçlar ve öneriler,")
print(" işletmenizin müşteri deneyimini iyileştirmek ve")
print(" büyüme stratejilerinizi desteklemek için kullanılabilir.")
print("=" * 80)

In [ ]:
# Final sonuçları kaydet
print("=== SONUÇLARI KAYDETME ===\n")

try:
    # Ana sonuçları kaydet
    rfm_scaled.to_csv('data/final_rfm_with_segments.csv', index=False)
    print("RFM verisi segmentlerle birlikte kaydedildi: data/final_rfm_with_segments.csv")
    
    # Model sonuçlarını kaydet
    results_df.to_csv('data/model_performance_results.csv', index=False)
    print("Model performans sonuçları kaydedildi: data/model_performance_results.csv")
    
    class_results_df.to_csv('data/classification_results.csv', index=False)
    print(" Sınıflandırma sonuçları kaydedildi: data/classification_results.csv")
    
    # Feature importance'ı kaydet
    feature_importance.to_csv('data/feature_importance.csv', index=False)
    print("Feature importance kaydedildi: data/feature_importance.csv")
    
    # Segment analizini kaydet
    cluster_analysis.to_csv('data/cluster_analysis.csv')
    print(" Cluster analizi kaydedildi: data/cluster_analysis.csv")
    
    # Günlük satış verilerini kaydet
    daily_sales.to_csv('data/daily_sales_data.csv', index=False)
    print(" Günlük satış verileri kaydedildi: data/daily_sales_data.csv")
    
    print(" Tüm sonuçlar 'data/' klasörüne kaydedildi.")
    
except Exception as e:
    print(f" Kaydetme sırasında hata: {e}")
    
print("\n" + "="*50)
print("JUPYTER NOTEBOOK SONUNDA!")
print("Tüm analizler tamamlandı ve sonuçlar kaydedildi.")
print(" Elde edilen insights'ları iş stratejilerinizde kullanabilirsiniz.")
print("="*50)